# Workflow Description

Ce notebook Jupyter documente le processus de simulation de données génétiques et de survie, ainsi que l'analyse de ces données à l'aide de la bibliothèque Hail et du modèle de risques proportionnels de Cox.

1. **Simulation des données VCF**:
    - Les données VCF sont simulées avec des génotypes pour 10 échantillons.
    - Le fichier VCF est créé et sauvegardé sous le nom `simulated.vcf`.

2. **Importation et filtrage des données VCF avec Hail**:
    - Les données VCF sont importées dans Hail.
    - Les lignes sont filtrées pour ne conserver que celles correspondant à l'identifiant `rs12345`.
    - Les entrées sont annotées avec le nombre d'allèles de risque.

3. **Simulation des données de survie**:
    - Les données de survie sont simulées pour les 10 échantillons, incluant le temps et l'événement.

4. **Combinaison des données de survie et des données génétiques**:
    - Les données de survie et les données de risque génétique sont combinées en un seul DataFrame.

5. **Analyse des données avec le modèle de Cox**:
    - Le modèle de risques proportionnels de Cox est ajusté aux données combinées.
    - Le résumé du modèle est affiché pour interprétation.

Les variables clés utilisées dans ce notebook incluent:
- `sample_names`: Liste des noms des échantillons.
- `genotypes`: Liste des génotypes simulés.
- `vcf_header` et `vcf_body`: Contenu du fichier VCF simulé.
- `vcf_filename`: Nom du fichier VCF.
- `df_risk`: DataFrame contenant les comptes d'allèles de risque par échantillon.
- `survival_data`: DataFrame contenant les données de survie simulées.
- `combined_df`: DataFrame combinant les données de survie et les données de risque génétique.

In [ ]:
import random
import os
import numpy as np
import pandas as pd

random.seed(42)
np.random.seed(42)

sample_names = [f"sample_{i}" for i in range(1, 11)]
genotypes = ["0/0", "0/1", "1/1", "0/1", "0/0", "1/1", "0/0", "0/1", "0/1", "1/1"]

vcf_header = (
    "##fileformat=VCFv4.2\n"
    "##source=DeepVariant\n"
    "#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t" + "\t".join(sample_names) + "\n"
)
vcf_body = "1\t10000\trs12345\tA\tG\t50\tPASS\t.\tGT\t" + "\t".join(genotypes) + "\n"

vcf_filename = "simulated.vcf"
with open(vcf_filename, "w") as f:
    f.write(vcf_header)
    f.write(vcf_body)

print("Simulated VCF (DeepVariant output) created:")
with open(vcf_filename, "r") as f:
    print(f.read())


In [ ]:
import hail as hl
hl.init(log='/tmp/hail.log', quiet=True)

mt = hl.import_vcf(vcf_filename, force_bgz=False)

mt = mt.filter_rows(mt.rsid == "rs12345")

mt = mt.annotate_entries(risk = mt.GT.n_alt_alleles())

df_risk = mt.entries().select('s', 'risk').to_pandas()
df_risk = df_risk.rename(columns={'s': 'sample'})
print("\nRisk allele counts from Hail (per sample):")
print(df_risk)


In [ ]:

survival_data = pd.DataFrame({
    "sample": sample_names,
    "time": np.random.exponential(scale=10, size=len(sample_names)),
    "event": np.random.binomial(1, 0.7, size=len(sample_names))
})
print("\nSimulated survival data:")
print(survival_data)


In [ ]:

combined_df = pd.merge(survival_data, df_risk, on="sample")
print("\nCombined survival & genetic data:")
print(combined_df)


In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(combined_df, duration_col='time', event_col='event', formula="risk")
print("\nCox Proportional Hazards model summary:")
cph.print_summary()
